In [38]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [58]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("EverGuide")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH00-Capstone


## 오픈소스 LLM 실행

In [39]:
query = "65세 여성 노인이 생활자금 관련해서 제주도에서 지원받을 수 있는 정책은 뭐가 있어? 지원받을 수 있는 정책을 다 알려줘"

In [40]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0-Q8_0:latest")

## RAG_PDF Process

In [41]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [42]:
loader = TextLoader('../../data/converted_markdown.txt')
data = loader.load()

print(f'type : {type(data)} / len : {len(data)}')
print(f'data : {data}')
print(f'page_content : {data[0].page_content}')

type : <class 'list'> / len : 1
data : [Document(metadata={'source': '../../data/converted_markdown.txt'}, page_content='<wantedList>\n**Total Count: **406\n\n**Page Number: **1\n\n**Number of Rows: **45000\n\n<resultCode>0</resultCode>\n<resultMessage>SUCCESS</resultMessage>\n\n### Department: 경상북도 구미시 구미보건소 건강증진과 치매안심센터\n\n**Province: **경상북도\n\n**Inquiry Number: **160\n\n**Theme: **신체건강\n\n**Last Modified: **20240820\n\n**Life Stage: **노년\n\n**Service Summary: **치매환자의 상태에 따라 돌봄에 필요한 조호물품을 무상공급하여 치매환자 및 가족의 경제적 부담을 경감.\n\n**More Info: **https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00005209&wlfareInfoReldBztpCd=02\n\n**Service ID: **WLF00005209\n\n**Service Name: **조호물품제공\n\n**Region: **구미시\n\n**Support Cycle: **분기\n\n**Provision Type: **현물지급\n\n**Target Audience: **저소득\n\n\n\n**Application Method: **방문\n\n### Department: 전라남도 나주시 복지환경국 복지정책과\n\n**Province: **전라남도\n\n**Inquiry Number: **501\n\n**Theme: **서민금융\n\n**Last Modified: **20240809\n\n*

In [43]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = CharacterTextSplitter(separator="###", chunk_size=800, chunk_overlap=100)
split_documents = text_splitter.split_documents(data)
print(f"분할된 청크의수: {len(split_documents)}")

Created a chunk of size 1081, which is longer than the specified 800
Created a chunk of size 867, which is longer than the specified 800


분할된 청크의수: 405


In [44]:
print(split_documents[100].page_content)

Department: 강원특별자치도 철원군 주민생활지원과

**Province: **강원특별자치도

**Inquiry Number: **145

**Last Modified: **20240629

**Life Stage: **노년

**Service Summary: **저소득 노인세대 건강보험료 납부로 건강증진에 기여

**More Info: **https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00003298&wlfareInfoReldBztpCd=02

**Service ID: **WLF00003298

**Service Name: **저소득노인가구 국민건강보험료 지원

**Region: **철원군

**Support Cycle: **월

**Provision Type: **기타



**Application Method: **방문


In [45]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

In [46]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [53]:
retriever = vectorstore.as_retriever()

In [56]:
# 원하는 문서 개수를 `k`에 지정하여 호출
document_list = retriever.invoke(query, k=10)  # 예: 상위 10개의 문서를 반환

In [58]:
# Document 객체의 page_content 내용만 빼서 하나의 문자열로 만드는 과정 함수화
def format_doc(document_list):
    return "\n\n\n".join([doc.page_content for doc in document_list])

In [69]:
document_list

[Document(metadata={'source': '../../data/converted_markdown.txt'}, page_content='Department: 제주특별자치도 제주시 복지위생국 노인복지과\n\n**Province: **제주특별자치도\n\n**Inquiry Number: **218\n\n**Theme: **서민금융\n\n**Last Modified: **20240628\n\n**Life Stage: **노년\n\n**Service Summary: **※ 제주시에 주소를 두고 실제로 거주하는 65세 이상인 노인장기요양보험법에 따른 장기요양수급자로 판정을 받지 못한 등급 외의 자(국민건강보험공단에서 정한 등급외 A,B 판정을 받은자) -노인의 안전한 보행과 거주하고 있은 주택의 낙상 예방 복지용구 비용을 지원하여 노후 일상생활의 안전을 도모 \n\n**More Info: **https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00004337&wlfareInfoReldBztpCd=02\n\n**Service ID: **WLF00004337\n\n**Service Name: **노인 생활안전사고 예방 지원\n\n**Region: **제주시\n\n**Support Cycle: **1회성\n\n**Provision Type: **현금지급\n\n\n\n**Application Method: **방문'),
 Document(metadata={'source': '../../data/converted_markdown.txt'}, page_content='Department: 제주특별자치도 제주시 복지위생국 노인복지과\n\n**Province: **제주특별자치도\n\n**Inquiry Number: **306\n\n**Theme: **서민금융, 일자리\n\n**Last Modified: **20240628\n\n**Life Stage: **노년\n\n**S